In [124]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import plotly.express as px
from datetime import timedelta
import requests
import pandas as pd
from geojson import Polygon
import geopandas as gpd
import h3
from datetime import datetime, timedelta
from google.cloud import storage
from google.colab import files

In [2]:
project = 'pelagic-rig-372910' 
location = 'US'
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [84]:
"""we can get dates from by using from datetime import date today = date.today()
but i will use an input """

get_date = '2014-03-02'

In [78]:
next_day = datetime.strptime(get_date, "%Y-%m-%d") + timedelta(days=1)

In [89]:
next_day = next_day.strftime("%Y-%m-%d")

In [90]:
daily_sql_query = """
            SELECT 
            vendor_id, store_and_fwd_flag, rate_code, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, airport_fee, total_amount,payment_type,
            distance_between_service, time_between_service, trip_type, imp_surcharge, pickup_location_id, dropoff_location_id, data_file_year, data_file_month, pickup_datetime, dropoff_datetime,
            FORMAT_DATE ('%d', pickup_datetime) as pickup_day, 
            FORMAT_DATE ('%m', pickup_datetime) as pickup_month,
            FORMAT_DATE ('%Y', pickup_datetime) as pickup_year,
            FORMAT_DATE ('%T', pickup_datetime) as pickup_time,

            FORMAT_DATE ('%d', dropoff_datetime) as dropoff_day, 
            FORMAT_DATE ('%m', dropoff_datetime) as dropoff_month,
            FORMAT_DATE ('%Y', dropoff_datetime) as dropoff_year,
            FORMAT_DATE ('%T', dropoff_datetime) as dropoff_time,

            FORMAT_DATE ('%F', pickup_datetime) as pickup_time,
            FORMAT_DATE ('%F', dropoff_datetime) as dropoff_time,
            FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2014` 
            WHERE  
                pickup_datetime >= '{date}' AND pickup_datetime < '{second_date}'
            ORDER BY pickup_datetime
            """.format(date = get_date, second_date = next_day)

In [92]:
sql_query = """
            SELECT 
            vendor_id, store_and_fwd_flag, rate_code, passenger_count, trip_distance, fare_amount, extra, mta_tax, tip_amount, tolls_amount, ehail_fee, airport_fee, total_amount,payment_type,
            distance_between_service, time_between_service, trip_type, imp_surcharge, pickup_location_id, dropoff_location_id, data_file_year, data_file_month, pickup_datetime, dropoff_datetime,
            FORMAT_DATE ('%d', pickup_datetime) as pickup_day, 
            FORMAT_DATE ('%m', pickup_datetime) as pickup_month,
            FORMAT_DATE ('%Y', pickup_datetime) as pickup_year,
            FORMAT_DATE ('%T', pickup_datetime) as pickup_time,

            FORMAT_DATE ('%d', dropoff_datetime) as dropoff_day, 
            FORMAT_DATE ('%m', dropoff_datetime) as dropoff_month,
            FORMAT_DATE ('%Y', dropoff_datetime) as dropoff_year,
            FORMAT_DATE ('%T', dropoff_datetime) as dropoff_time,

            FORMAT_DATE ('%F', pickup_datetime) as pickup_time,
            FORMAT_DATE ('%F', dropoff_datetime) as dropoff_time,
            FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2014` 
            WHERE  
                pickup_datetime >= '2014-03-01' AND pickup_datetime < '2014-03-08'
            ORDER BY pickup_datetime
            """

In [82]:
"""If the process needs to run everyday, i will be using daily_sql_query otherwise i will keep sql_query"""

'If the process needs to run everyday, i will be using daily_sql_query otherwise i will keep sql_query'

In [4]:
"""Checking the dataframe through job id i have created my query."""

#job = client.get_job('bquxjob_4517eb0_1856c2ab54f')
job = client.query(sql_query)
taxi_df = job.to_dataframe()

In [5]:
"""i will be converting this values to numeric to make calculations trip_distance, fare_amount, total_amount"""

for k in ["trip_distance", "fare_amount", "total_amount", "tolls_amount"]:
  taxi_df[k] = taxi_df[k].astype(float)


In [6]:
"""checking if there is any passenger_count == 0 and returning that index
if there is any passenger_count == 0, it could be because there was no passenger or there is an error in that specific data.

passenger_count, trip_distance, fare_amount, total_amount
"""

def check_zeros(df_name, column_name):
  return df_name[df_name[column_name].isin([0])].index


In [7]:
"dropping 0 values"
fields_to_check = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]

for k in fields_to_check:
  taxi_df = taxi_df.drop(check_zeros(taxi_df, k))

In [8]:
taxi_df.describe()[fields_to_check]

,passenger_count,trip_distance,fare_amount,total_amount
count,277780.000000,277780.000000,277780.000000,277780.000000
mean,1.412229,2.871011,11.929553,13.775742
std,1.113904,2.772733,8.470145,9.658673
min,1.000000,0.010000,0.010000,0.010000
25%,1.000000,1.100000,6.500000,7.500000
50%,1.000000,1.940000,9.500000,10.500000
75%,1.000000,3.690000,15.000000,17.000000
max,8.000000,101.260000,432.000000,433.500000


In [9]:
"""In here i will be visualizing some values to see if there are any other outliers."""
"this does not follow normal distribution so i will see with a box plot"
px.histogram(taxi_df, x = "fare_amount") 



In [10]:
px.box(taxi_df, x = "fare_amount")

In [11]:
"""by looking through nyc.gov taxi fares i noticed that intial charge is 2.50$ and 70 cent for 1/5 miles. 3.50$ more for every mile.
so for every mile i get 3.50$ + 3$
"""

'by looking through nyc.gov taxi fares i noticed that intial charge is 2.50$ and 70 cent for 1/5 miles. 3.50$ more for every mile.\nso for every mile i get 3.50$ + 3$\n'

In [12]:
"""now i wanna see if trip distance is correlated correctly with fare amount.
i will write a function which takes trip distance multiplies it by 3.50 and adds up 2.50$ dollars to see how much approximately it costs
and add up some value to see if it is between that threshold. if it is not i will store them in a list.
"""

def is_fee_acceptable(x, amount):
  """i am adding extra **A**amount dollars to see how it goes"""
  sus_indexes = []
  for k in x[["trip_distance", "fare_amount", "tolls_amount"]].iterrows():
    normal_fee = (k[1]["trip_distance"] * 2.50) + 3
    if ((normal_fee - amount) + k[1]["tolls_amount"] <= k[1]["fare_amount"] and (normal_fee + amount) + k[1]["tolls_amount"]  >= k[1]["fare_amount"]) == True:
      pass
    else:
      sus_indexes.append(k[0])

  return sus_indexes



In [13]:
len(taxi_df)

277780

In [14]:
"i decided to set extra amount rate as 15 and drop rest of the data"
taxi_df = taxi_df.drop(is_fee_acceptable(taxi_df, 15))

In [15]:
len(taxi_df)

276531

i will be dropping if fare amount is less than 2.5 (initial charge)

In [16]:
taxi_df = taxi_df.drop(taxi_df[taxi_df["fare_amount"] < 2.5].index)

In [17]:
len(taxi_df)

276499

In [18]:
"checking travel time and if it is greater than 1 minute i will be keeping it."
def check_travel_time(x):
  index = x[0]
  pickup = x[1]["pickup_datetime"]
  dropoff = x[1]["dropoff_datetime"]
  if dropoff - pickup <  timedelta(days=0, minutes=1):
    return index
  else:
    pass

In [19]:
time_problem = [(check_travel_time(k)) for k in taxi_df[["pickup_datetime", "dropoff_datetime"]].iterrows()]

In [20]:
time_problem = [k for k in time_problem if k != None]

In [21]:
taxi_df = taxi_df.drop(time_problem)

In [22]:
len(taxi_df)

274671

In [23]:
taxi_df["trip_duration"] =  taxi_df["dropoff_datetime"] - taxi_df["pickup_datetime"]

In [24]:
"""there could be still more outliers i wanna see if i can find them """

'there could be still more outliers i wanna see if i can find them '

In [25]:
def outlier_iqr(data):
  quarter_1 = data.quantile(0.25)
  quarter_3 = data.quantile(0.75)
  iqr = quarter_3 - quarter_1
  outliers = data[((data<(quarter_1 -1.5 * iqr)) | (data>(quarter_3 + 1.5 * iqr)))]
  return outliers

In [26]:
outliers = outlier_iqr(taxi_df["fare_amount"])

In [27]:
"""for this time i wanted to set a max and min limit and stack values between them instead of dropping them"""

'for this time i wanted to set a max and min limit and stack values between them instead of dropping them'

In [28]:
#px.box(x = outliers)

In [29]:
def day_part(x):
  if x >= "06:00:00" and x <= "11:59:59":
    return "Morning"
  elif x >= "12:00:00" and x <= "17:59:59":
    return "Noon"
  elif x >= "18:00:00" and  x <= "23:59:59":
    return "Evening"
  elif x >= "00:00:00" and x <= "05:59:59":
    return "Night"

In [30]:
taxi_df["dropoff_daypart"] = [day_part(k) for k in taxi_df["dropoff_time"]]
taxi_df["pickup_daypart"] = [day_part(k) for k in taxi_df["pickup_time"]]

In [31]:
"""So my data did not have any locational information other than dropoff and pickup location ids. I found out those locations match with their coordinates. I will be connecting to an API and importing those location and creating some fields in my data"""

'So my data did not have any locational information other than dropoff and pickup location ids. I found out those locations match with their coordinates. I will be connecting to an API and importing those location and creating some fields in my data'

In [32]:
response = requests.get("https://data.cityofnewyork.us/resource/755u-8jsi.json")

#response = requests.get("https://data.cityofnewyork.us/resource/2np7-5jsg.json")

In [33]:
coordinated_data = pd.read_json(response.text)

In [34]:
def get_geom(input, df):
  input = int(input)
  try:
    return df[df["location_id"].isin([input])]["the_geom"].to_list()[0]
  except:
    return None



def get_zone_info(input,df):
  input = int(input)
  try:
    return df[df["location_id"].isin([input])]["zone"].to_list()[0]
  except:
    return None


In [35]:
taxi_df["pick_up_location_geometry"] = [get_geom(k, coordinated_data) for k in taxi_df["pickup_location_id"]]

In [36]:
taxi_df["drop_off_location_geometry"] = [get_geom(k, coordinated_data) for k in taxi_df["dropoff_location_id"]]

In [37]:
taxi_df["pick_up_zone"] = [get_zone_info(k, coordinated_data) for k in taxi_df["pickup_location_id"]]

In [38]:
taxi_df["drop_off_zone"] = [get_zone_info(k, coordinated_data) for k in taxi_df["dropoff_location_id"]]

In [39]:
"there are outof newyork pickup and drop off zones. i will be dropping them too."
out_of_ny_pickup = taxi_df[taxi_df["pick_up_zone"].isin([None])].index
out_of_ny_dropoff = taxi_df[taxi_df["drop_off_zone"].isin([None])].index

In [40]:
taxi_df = taxi_df.drop(out_of_ny_pickup)
out_of_ny_dropoff = [k for k in out_of_ny_dropoff if k not in out_of_ny_pickup]
taxi_df = taxi_df.drop(out_of_ny_dropoff)

In [41]:
"""since my dataset does not have lat or longs, i have multi_polygons i will be converting multi_polygons to polygons then use h3."""

'since my dataset does not have lat or longs, i have multi_polygons i will be converting multi_polygons to polygons then use h3.'

In [42]:
import copy
def convert_format(x):
    y = copy.deepcopy(x)
    y["coordinates"] = y["coordinates"][0]
    y["coordinates"] = list(y["coordinates"])
    y["type"] = "Polygon"
    hexes = h3.polyfill(y, 9)
    return hexes

In [43]:
taxi_df["pick_up_hexes"] = [convert_format(k) for k in taxi_df["pick_up_location_geometry"]]

In [44]:
taxi_df["drop_off_hexes"] = [convert_format(k) for k in taxi_df["drop_off_location_geometry"]]

In [45]:
"most popular pickup hexagons"

'most popular pickup hexagons'

In [46]:
pickup_hexes_count = {}
for k,y in zip(taxi_df["pick_up_hexes"], taxi_df["pickup_daypart"]):
  for element in k:
    if element not in pickup_hexes_count.keys():
      pickup_hexes_count[element] = [0, y]
    elif element in pickup_hexes_count.keys():
      pickup_hexes_count[element][0] += 1
    else:
      print(k, "Problem")

In [47]:
pickup_hexes_df = pd.DataFrame(columns = ["pick_up_hexes", "count", "day_time"])
for k in pickup_hexes_count.items():
  pickup_hexes_df = pickup_hexes_df.append({"pick_up_hexes": k[0], "count": k[1][0], "day_time": k[1][1]}, ignore_index = True)

In [105]:
pickup_hexes_df = pickup_hexes_df.sort_values(by=["count"], ascending = False)

In [49]:
"""most popular dropoff hexagons"""

'most popular dropoff hexagons'

In [117]:
dropoff_hexes_count = {}
for k,y in zip(taxi_df["drop_off_hexes"], taxi_df["dropoff_daypart"]):
  for element in k:
    if element not in dropoff_hexes_count.keys():
      dropoff_hexes_count[element] = [0, y]
    elif element in dropoff_hexes_count.keys():
      dropoff_hexes_count[element][0] += 1
    else:
      print(k, "Problem")

In [118]:
drop_off_hexes_df = pd.DataFrame(columns = ["drop_off_hexes", "count", "day_time"])
for k in dropoff_hexes_count.items():
  drop_off_hexes_df = drop_off_hexes_df.append({"drop_off_hexes": k[0], "count": k[1][0], "day_time": k[1][1]}, ignore_index = True)

In [119]:
drop_off_hexes_df = drop_off_hexes_df.sort_values(by=["count"], ascending = False)

In [53]:
"""most popular routes"""

'most popular routes'

In [55]:
routes_hexes_count = {}

In [56]:
for k,y in zip(taxi_df["pick_up_hexes"], taxi_df["pickup_daypart"]):
  route = '**'.join(map(str,list(k)))
  if route not in routes_hexes_count.keys():
    routes_hexes_count[route] = [0, y]
  elif route in routes_hexes_count.keys():
    routes_hexes_count[route][0] += 1
  else:
    print(k, "Problem")

In [57]:
"""##no need for drop_off it is just the same way reversed.##

for k,y in zip(taxi_df["drop_off_hexes"], taxi_df["dropoff_daypart"]):
  route = '**'.join(map(str,list(k)))
  if route not in routes_hexes_count.keys():
    routes_hexes_count[route] = [0, y]
  elif route in routes_hexes_count.keys():
    routes_hexes_count[route][0] += 1
  else:
    print(k, "Problem")"""

'##no need for drop_off it is just the same way reversed.##\n\nfor k,y in zip(taxi_df["drop_off_hexes"], taxi_df["dropoff_daypart"]):\n  route = \'**\'.join(map(str,list(k)))\n  if route not in routes_hexes_count.keys():\n    routes_hexes_count[route] = [0, y]\n  elif route in routes_hexes_count.keys():\n    routes_hexes_count[route][0] += 1\n  else:\n    print(k, "Problem")'

In [58]:
route_hexes_df = pd.DataFrame(columns = ["routes", "count", "day_time"])
for k in routes_hexes_count.items():
  route_hexes_df = route_hexes_df.append({"routes": k[0], "count": k[1][0], "day_time": k[1][1]}, ignore_index = True)

In [102]:
route_hexes_df = route_hexes_df.sort_values('count', ascending = False)

In [60]:
len(route_hexes_df)

217

In [114]:
pickup_hexes_df.to_csv("pickup_hexes_df.csv")
files.download(r"pickup_hexes_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [120]:
drop_off_hexes_df.to_csv("drop_off_hexes_df.csv")
files.download(r"drop_off_hexes_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
route_hexes_df.to_csv("route_hexes_df.csv")
files.download(r"route_hexes_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>